<h3 style="background-color: #D3D92B;"><br>Numerical Optimisation. Project 1<br></h3><br>

<h3 style="background-color: #D3D92B;"><br>Team Information<br></h3><br>

<i>Group 7<br>
Participants information in alphabetical order</i>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Alexander</td>
    <td style = "text-align: left">Mair</td>
    <td style = "text-align: left">k11916624</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Andreas</td>
    <td style = "text-align: left">Oberdammer</td>
    <td style = "text-align: left">k11908776</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Dominik</td>
    <td style = "text-align: left">Zauner</td>
    <td style = "text-align: left">k11717988</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Florian</td>
    <td style = "text-align: left">Rothkegel</td>
    <td style = "text-align: left">k11908775</td>
  </tr>
  <tr>
    <td style = "text-align: left">5</td>
    <td style = "text-align: left">Martin</td>
    <td style = "text-align: left">Stockinger</td>
    <td style = "text-align: left">k01035089</td>
  </tr>
  <tr>
    <td style = "text-align: left">6</td>
    <td style = "text-align: left">Martin</td>
    <td style = "text-align: left">Zwiffl</td>
    <td style = "text-align: left">k11910668</td>
  </tr>

</table>

<h3 style="background-color: #D3D92B;"><br>Implementation<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Imports<br></h4><br>
<i>Describe how to install additional packages, if you have some, here</i><br><br>

<b>No additional libaries are needed/installed</b>



In [1]:
import numpy as np
import scipy 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

<hr><h4 style="background-color: #ADB8FF;"><br>Stopping criteria<br></h4><br>
<i>Place for additional comments and argumentation</i><br><br>
We implemented the stopping criterion based on the hints given in the second phase PDF. Description of the funciton can be found in the code

In [2]:
#your function for stopping criterium
def stopping_criteria(x_start, x_curr, x_prev, x_after_start, fn, gradient_fn, iteration, tolerance=1e-7, max_steps=1e6):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_start: Start point coordinates. Postion on which the algorithm should start
    --------------------------------------------------------------------------------------------------------------
    x_curr: Current point coordinates. Postion on which the algorithm is currently
    --------------------------------------------------------------------------------------------------------------
    x_prev: Previous point coordinates. Postion on which the algorithm is was in the step before
    --------------------------------------------------------------------------------------------------------------
    x_after_start: Fist calculated point coordinates. Postion on which the algorithm was after the first step
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    iteration: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    tolerance: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    max_steps: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    - this function checks if the gradient of the current point is below the tolerance
    - Additionally it checks if the algo exceeds the allowed max steps
    - It is also checked, if the gradient of the current step is smaller than the gradient in the start point
      times our tolerance. In our case we noticed by testing, that the algorithm stops a bit too soon, therefore
      we introduced a correction factor, which additionally lowers the error bound. Again --> Payoff runtime/acc
    - As mentioned in the second phase pdf, we can make the stopping crit relative, by comparing it to the 
      tolerance multiplied with the difference from the start to the first point. this can be done for f(x) and x.
    """
    correction_factor = 1e-3 # factor to adjust error bound and gain more precicsion (later stopping). Explained above.
    
    if np.linalg.norm(gradient_fn(x_curr)) <= tolerance:
        print("*** GRADIENT BELOW TOLERANCE ***")
        return True
    
    if iteration >= max_steps:
        print("*** MAX STEPS EXCEEDED ***")
        return True
    
    if np.linalg.norm(gradient_fn(x_curr)) <= tolerance * np.linalg.norm(gradient_fn(x_start)) * correction_factor:
        print("*** GRADIENT SMALLER THAN GRADIENT OF FIRST STEP * TOLERANCE ***")
        return True
    
    if x_prev is not None and x_after_start is not None:
        x_diff_first_step = np.linalg.norm(x_after_start - x_start)
        y_diff_first_step = np.abs(fn(x_after_start) - fn(x_start))
        
        if np.abs(fn(x_curr) - fn(x_prev)) <= tolerance * y_diff_first_step * correction_factor:
            print("*** Y DIFFERENCE SMALLER THAN ADJUSTED TOLERANCE ***")
            return True
        
        if np.linalg.norm(x_curr - x_prev) <= tolerance * x_diff_first_step * correction_factor:
            print("*** X DIFFERENCE SMALLER THAN ADJUSTED TOLERANCE ***")
            return True
    

<hr><h4 style="background-color: #ADB8FF;"><br>Varibales scaling<br></h4><br>
<i>Place your reasoning here, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.</i><br><br>
    
In comparison to gradient descent, Newton's method is affine invariant. Given $f$, nonsingular $A \in \mathbb{R}^{n \times n}$. Let $x=A y$, and $g(y)=f(A y)$. Newton steps on $g$ are: 

\begin{aligned}
y^{+} &=y-\left(\nabla^{2} g(y)\right)^{-1} \nabla g(y) \
&=y-\left(A^{T} \nabla^{2} f(A y) A\right)^{-1} A^{T} \nabla f(A y) \
&=y-A^{-1}\left(\nabla^{2} f(A y)\right)^{-1} \nabla f(A y)
\end{aligned}

Hence
$$
A y^{+}=A y-\left(\nabla^{2} f(A y)\right)^{-1} \nabla f(A y)
$$
i.e.
$$
x^{+}=x-\left(\nabla^{2} f(x)\right)^{-1} f(x)
$$
    
So progress is independent of problem scaling.

<hr><h4 style="background-color: #ADB8FF;"><br>Stabilising algorithm<br></h4><br>
<i>Place your reasoning here, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.</i>
<br><br>
Stability is a really important topic for those type of algorithms, because if the stability is not given, it is not ensured that we even reach our goal, because the algorithm could do basically anything and even crash.<br>
Therefore it is really important to take some measures to ensure that the algorithm is stable and works fine.<br><br>

Possible Measures:<br>
- Avoid floating points problems and round off errors<br>
- Scale problems if necessary<br>
- Avoid matrix inversion<br>
- Use optimal hyperparamters for algorithm<br>
- Improve stopping criterion (limit max steps, verify multiple attributes)<br><br>

We implemented some of those measure. We tuned our tolerance (epsilon) and the step size to avoid overshooting of the algorithm. We also use float calculation to avoid big round off errors (read more in floating point section).<br>
We also use LU decomposition to avoid direct matrix inversion and we also improved our stopping criterion. These measure have a positive impact on the stability of our algorithm. <br>
<br>
There are for sure more stability improvements, but as we can see on our tests, the algorithm is performing reasonable well and therefore we did not implement more features. <br><br>

<b>Below we want to show a described problem, if we would not use default float arrays for our calculation.</b><br>
You should be able to see that the numpy result (which is correct result) is equal to the result, if we typecast the int array to an float array. If we just use the int array, we get wrong results and this destroys the stability of the aglo and makes it nearly unuseable.
<br><b>Therefore is the useage of float a major contribution for stability </b>

In [3]:
#your function for stabilising goes here
def lu_deco_inverse_not_float(A):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    A: Input matrix, from which the inverse has to be calculated 
    --------------------------------------------------------------------------------------------------------------
    - LU Decompostion matrix factorization, creates two trinangular matrices L and U. In this case this is a bit
      simpler. We create a permuation matrix and a matrix (a), which is a combination of L and U.
    - Based on this matrices we are able to create the inverted version of the given matrix
    - Float conversion is missing in this function to show the problems, which happen with no typecasting and int
      calculation
    """
    n = A.shape[0]
    a = np.copy(A)
    p = np.eye(n)

    for i in range(n - 1):
        j = i + np.argmax(np.abs(a[i:, i]))  # maximum in column
        if np.abs(a[i, j]) < 1e-8:
            print("Matrix is singular")
            sys.exit()

        if i != j:
            a[[i, j]] = a[[j, i]]
            p[[i, j]] = p[[j, i]]

        for i_row in range(i + 1, n):
            a[i_row, i] = a[i_row, i] / a[i, i]
        for i_row in range(i + 1, n):
            for i_col in range(i + 1, n):
                a[i_row, i_col] = a[i_row, i_col] - np.outer(a[i_row, i], a[i, i_col])

    a_inv = np.copy(np.dot(p, np.eye(n)))

    for i in range(n):
        for j in range(i):
            a_inv[i] -= np.dot(a[i, j], a_inv[j])  # Forward substitution

    for i in reversed(range(n)):
        for j in range(i + 1, n):
            a_inv[i] = a_inv[i] - np.dot(a[i, j], a_inv[j])  # Backward substitution
        a_inv[i] = a_inv[i] / a[i, i]

    return a_inv

def lu_deco_inverse_float(A):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    A: Input matrix, from which the inverse has to be calculated 
    --------------------------------------------------------------------------------------------------------------
    - LU Decompostion matrix factorization, creates two trinangular matrices L and U. In this case this is a bit
      simpler. We create a permuation matrix and a matrix (a), which is a combination of L and U.
    - Based on this matrices we are able to create the inverted version of the given matrix
    - Float conversion is done here to get the correct results (no round of error/auto typecasting problem)
    """
    n = A.shape[0]
    a = np.copy(A.astype(float))
    p = np.eye(n)

    for i in range(n - 1):
        j = i + np.argmax(np.abs(a[i:, i]))  # maximum in column
        if np.abs(a[i, j]) < 1e-8:
            print("Matrix is singular")
            sys.exit()

        if i != j:
            a[[i, j]] = a[[j, i]]
            p[[i, j]] = p[[j, i]]

        for i_row in range(i + 1, n):
            a[i_row, i] = a[i_row, i] / a[i, i]
        for i_row in range(i + 1, n):
            for i_col in range(i + 1, n):
                a[i_row, i_col] = a[i_row, i_col] - np.outer(a[i_row, i], a[i, i_col])

    a_inv = np.copy(np.dot(p, np.eye(n)))

    for i in range(n):
        for j in range(i):
            a_inv[i] -= np.dot(a[i, j], a_inv[j])  # Forward substitution

    for i in reversed(range(n)):
        for j in range(i + 1, n):
            a_inv[i] = a_inv[i] - np.dot(a[i, j], a_inv[j])  # Backward substitution
        a_inv[i] = a_inv[i] / a[i, i]

    return a_inv

test_mat = np.array([[2,-1,0], [1,2,-2], [0,-1,1]]) # matrix to test scenario on. Can be chosen abitrarly

print("============================================================================")
print("MATRIX INVERSION WITH AND WITHOUT FLOAT CALCULATION")
print("============================================================================")
print("INVERSE MATRIX WITHOUT PRE FLOAT TYPECASTING")
print(lu_deco_inverse_not_float(test_mat))
print("============================================================================")
print("INVERSE MATRIX WITH PRE FLOAT TYPECASTING")
print(lu_deco_inverse_float(test_mat))
print("============================================================================")
print("INVERSE MATRIX NUMPY CALCULATION (TRUE RESULT)")
print(np.linalg.inv(test_mat))

MATRIX INVERSION WITH AND WITHOUT FLOAT CALCULATION
INVERSE MATRIX WITHOUT PRE FLOAT TYPECASTING
[[0.5  0.25 0.5 ]
 [0.   0.5  1.  ]
 [0.   0.   1.  ]]
INVERSE MATRIX WITH PRE FLOAT TYPECASTING
[[-1.11022302e-16  1.00000000e+00  2.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  4.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  5.00000000e+00]]
INVERSE MATRIX NUMPY CALCULATION (TRUE RESULT)
[[-1.11022302e-16  1.00000000e+00  2.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  4.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  5.00000000e+00]]


<hr><h4 style="background-color: #ADB8FF;"><br>Fighting floating-point numbers and roundoff error<br></h4><br>
<i>Place your reasoning, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.</i>
<br><br>
In our algorithm and calculations we only use floating point numbers. There is always the risk of roundoff errors and therefore loss in accuracy, which leads to increased error rates.<br>
We are fully aware of this problem and therefore suggest the following options:<br>
- Do every calculation in float, in order to avoid typecasting and high round of errors<br>
- Increase precision of float (assign more memory (bits)) to the variables and reduce error (increased memory usage + computaton time)<br>
- We could also use a special python libary for high precision calculation. Same as before, this increases memory usage and computation time.<br><br>

As we can see, there are some measure we can take, in order to avoid round off errors. The most important measure is the first one. If we calculate everything in float, the round off error is small enough <br>
and as we can see, based on our tests below, we do not have any problems, therefore this measure seems to be enough. If we really have the need for super high precision calculation we have to either increase the<br>
memory size of float or use an external libary

<hr><h4 style="background-color: #ADB8FF;"><br>Inverting matrices<br></h4><br>
<i>Place for additional comments and argumentation</i><br><br>
    
The algorithm of choice for inverting matrices is based on the approach from the book <b> Numerical Optimization</b> written by <b><i> Jorge Nocedal, Stephen J. Wright</i></b> and the calculator of TU Graz (http://lampx.tugraz.at/~hadley/num/ch2/2.3.php).<br>
We do not directly decompose the matrix into L and U, because this not really needed in our case, since we reuse the LU matrix and do not use the two parts separate. <i>a_inv</i> is basically the the LU matrix, which is than used to create inverse.
<br><br><b>IMPORTANT:</b> This is also mentioned in the floating point/error section. If the matrix is parsed with int values, python has to do typecasting, because of the divison, which is done in this function. As far as our testing goes, typecasting leads<br>
to an major error in calculation and leads to wrong inverse matrices. This is not only the case for this function, but also in some other cases we encountered this problem. For more details check the floating point/stability section

In [4]:
#your function for invertion goes here
def lu_deco_inverse(A):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    iteration: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    tolerance: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    max_steps: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    - this function checks if the gradient of the current point is below the tolerance
    - Additionally it checks if the algo exceeds the allowed max steps
    - It is also checked, if the gradient of the current step is smaller than the gradient in the start point
      times our tolerance. In our case we noticed by testing, that the algorithm stops a bit too soon, therefore
      we introduced a correction factor, which additionally lowers the error bound. Again --> Payoff runtime/acc
    - As mentioned in the second phase pdf, we can make the stopping crit relative, by comparing it to the 
      tolerance multiplied with the difference from the start to the first point. this can be done for f(x) and x.

    --------------------------------------------------------------------------------------------------------------
    - LU Decompostion matrix factorization, creates two trinangular matrices L and U. In this case this is a bit
      simpler. We create a permuation matrix and a matrix (a), which is a combination of L and U.
    - Based on this matrices we are able to create the inverted version of the given matrix
    - Float conversion is done here to get the correct results (no round of error/auto typecasting problem)
    """
    n = A.shape[0]
    a = np.copy(A.astype(float))
    p = np.eye(n)

    for i in range(n - 1):
        j = i + np.argmax(np.abs(a[i:, i]))  # maximum in column
        if np.abs(a[i, j]) < 1e-8:
            print("Matrix is singular")
            sys.exit()

        if i != j:
            a[[i, j]] = a[[j, i]]
            p[[i, j]] = p[[j, i]]

        for i_row in range(i + 1, n):
            a[i_row, i] = a[i_row, i] / a[i, i]
        for i_row in range(i + 1, n):
            for i_col in range(i + 1, n):
                a[i_row, i_col] = a[i_row, i_col] - np.outer(a[i_row, i], a[i, i_col])

    a_inv = np.copy(np.dot(p, np.eye(n)))

    for i in range(n):
        for j in range(i):
            a_inv[i] -= np.dot(a[i, j], a_inv[j])  # Forward substitution

    for i in reversed(range(n)):
        for j in range(i + 1, n):
            a_inv[i] = a_inv[i] - np.dot(a[i, j], a_inv[j])  # Backward substitution
        a_inv[i] = a_inv[i] / a[i, i]

    return a_inv

<hr><h4 style="background-color: #ADB8FF;"><br>Gradients calculation<br></h4><br>
<i>Place for additional comments and argumentation</i>
<br><br>  
As suggested, we now approximate derivative and hessians. We did this by using the trivial formulas of section 8.1 (central- difference). <br>
4 wrapper functions were created, because the different shapes, so we can use the aproximated derivatives, gradients, second_derivatives and hessians just as before.

In [5]:
#your function for gradient approximation goes here
def differential_quotient_helper(fn, x, epsilon=1e-9):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    x: point, which should be taken into account. Mostly used for the shape to recreate correct der/grad shape
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Algorithm, which approximates numericaly the gradient of a given function f
    """
    res=[]
    x = x.astype(float)
    x=x.reshape(x.shape[1] if len(x.shape) > 1 and x.shape[1] > x.shape[0] else x.shape[0])
    for idx in range(len(x)):
        direction = np.zeros(len(x))
        direction[idx] = 1
        part_derivative = (fn(x+epsilon*direction)-fn(x-epsilon*direction))/(2*epsilon)
        res.append(part_derivative)
    return np.array(res)

def grad_wrapper(fn,  epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable gradient function of a given problem function fn
    """
    return lambda x: differential_quotient_helper(fn, x,  epsilon)

def hessian_wrapper(fn, epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable hessian function of a given problem function fn
    """
    return lambda x: grad_wrapper(grad_wrapper(fn, epsilon), epsilon)(x).reshape(len(x),len(x))

def derivative_wrapper(fn, epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable derivaitve function of a given problem function fn
    """
    return lambda x: (fn(x+epsilon)-fn(x-epsilon))/(2*epsilon)

def second_derivative_wrapper(fn, epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable second derivative function of a given problem function fn
    """
    return lambda x: derivative_wrapper(derivative_wrapper(fn, epsilon), epsilon)(x)

<hr><h4 style="background-color: #ADB8FF;"><br>Additional objects you implemented<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [6]:
#your code goes here

<hr><h4 style="background-color: #ADB8FF;"><br>Optimising algorithm itself<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [7]:
#your code goes here
def newton_multi(x_start, fn, gradient_fn, hessian_fn, epsilon=1e-5,
         activate_new_stop_crit=False,
         activate_pre_scaling=False,
         activate_alt_gradient=False,
         activate_new_matrix_inversion=False):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_start: Start point coordinates. Postion on which the algorithm should start
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    hessian_fn: function, which is the hessian of the problem. 
                 One can parse coordinates to this function to get the hessian.  
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    activate_new_stop_crit: Boolean Flag, which de/activates the usage of the additinal stopping criterations.
                            If activated, the implemented stopping crit function is called and all features
                            get checked.
    --------------------------------------------------------------------------------------------------------------
    activate_pre_scaling: Boolean Flag, to activate pre scaling of the problem. In this case, we have not 
                          implemented pre scaling.
    --------------------------------------------------------------------------------------------------------------
    activate_alt_gradient: Boolean Flag, to activate alternative gradient calculation. This calls the gradient
                           wrapper and approximates the gradient function.
    --------------------------------------------------------------------------------------------------------------
    activate_new_matrix_inversion: Boolean Flag, to activate alternative matrix inversion. This calls the LU
                                   decomposition function, but is not needed in this algo since nothing needs to
                                   be inversed.
    --------------------------------------------------------------------------------------------------------------
    - newton method for quadractic multi variate examples
    """
    i = 0
    xk = x_start
    prev_x = None
    x_step_1 = None 
    
    if activate_alt_gradient:
        gradient_fn = grad_wrapper(fn)
        hessian_fn = hessian_wrapper(fn)
    
    while True:
        grad = gradient_fn(xk)
        grad_norm = np.linalg.norm(grad)

        if activate_new_stop_crit and stopping_criteria(x_start, xk, prev_x, x_step_1, fn, gradient_fn, i):
            break

        elif not activate_new_stop_crit and grad_norm <= epsilon:
            break
        
        prev_x = xk
        
        if not activate_new_matrix_inversion:
            xk=xk - (np.linalg.inv(hessian_fn(xk))@grad)
        elif activate_new_matrix_inversion:
            xk=xk - (lu_deco_inverse(hessian_fn(xk))@grad)
            
        if i == 0:
            x_step_1 = xk
            
        i+= 1
        
    print("performed " + str(i+1) + " iterations.")
    return xk

def newton_uni(x_start, fn, derivative_fn, second_derivative_fn, epsilon=1e-6,
         activate_new_stop_crit=False,
         activate_pre_scaling=False,
         activate_alt_gradient=False,
         activate_new_matrix_inversion=False):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_start: Start point coordinates. Postion on which the algorithm should start
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    activate_new_stop_crit: Boolean Flag, which de/activates the usage of the additinal stopping criterations.
                            If activated, the implemented stopping crit function is called and all features
                            get checked.
    --------------------------------------------------------------------------------------------------------------
    activate_pre_scaling: Boolean Flag, to activate pre scaling of the problem. In this case, we have not 
                          implemented pre scaling.
    --------------------------------------------------------------------------------------------------------------
    activate_alt_gradient: Boolean Flag, to activate alternative gradient calculation. This calls the gradient
                           wrapper and approximates the gradient function.
    --------------------------------------------------------------------------------------------------------------
    activate_new_matrix_inversion: Boolean Flag, to activate alternative matrix inversion. This calls the LU
                                   decomposition function, but is not needed in this algo since nothing needs to
                                   be inversed.
    --------------------------------------------------------------------------------------------------------------
    - newton method for quadractic multi variate examples
    """
    i = 0
    xk = x_start
    prev_x = None
    x_step_1 = None
    
    if activate_alt_gradient:
        derivative_fn = derivative_wrapper(fn)
        second_derivative_fn = second_derivative_wrapper(fn)
    
    while True:
        der = derivative_fn(xk)
        

        if activate_new_stop_crit and stopping_criteria(x_start, xk, prev_x, x_step_1, fn, derivative_fn, i):
            break

        elif not activate_new_stop_crit and abs(der) <= epsilon:
            break

        prev_x = xk
        xk=xk - der/second_derivative_fn(xk)
        
        if i == 0:
            x_step_1 = xk
        
        i+= 1
    print("performed " + str(i+1) + " iterations.")
    return xk

<h3 style="background-color: #D3D92B;"><br>Testing on 5-10 variables, Quadratic objective<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Implement functions to optimise over<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [8]:
#your code goes here
def get_random_A_b_minimizer(seed):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    seed: Seed is an int value, which is used to create reproducable random values.
    --------------------------------------------------------------------------------------------------------------
    - 10x10 random matrix is produced by this function, which is the base of the problem
    - 10x1 random vector is produced, which is also the minimizer of the function and the goal of the algo
    - Function uses thoes variables to create a problem function, which can be solved by the algo
    """
    np.random.seed(seed)
    A=np.random.randint(0, 2, (10, 10))
    A=A@A.T

    np.random.seed(seed)
    minimizer=np.random.randint(0, 10, (10, 1))

    b=A@minimizer

    return A, b, minimizer

# Created problem instances, which are used in the functions below, to provide a useable set of prolems for the algo, A and b correspond to function depended values and Minimizer is the solution to the problem
A_1, b_1, minimizer_1 = get_random_A_b_minimizer(0)
A_2, b_2, minimizer_2 = get_random_A_b_minimizer(2)
A_3, b_3, minimizer_3 = get_random_A_b_minimizer(3)
A_4, b_4, minimizer_4 = get_random_A_b_minimizer(12)
A_5, b_5, minimizer_5 = get_random_A_b_minimizer(15)

#############################################################################################################################################################
# problem_x_q(x) --> is the problem function f, which is used by algorithm.
# grad_problem_X_q(x) --> as the name implies, this function is the gradient of the problem function f (problem_X_q)
# shessian_problem_X_q(x) --> as the name implies, this function is the hessian of the problem function f (problem_X)
# in all cases you can parse x coordinates to the functions in order to receive the corresponding y axis values.
#############################################################################################################################################################
# Problem 1

def problem_1_q(x):
    return (x.T@A_1@x)/2-b_1.T@x

def grad_problem_1_q(x):
    return np.dot(A_1,x)-b_1

def hessian_problem_1_q(x):
    return A_1

#############################################################################################################################################################
# Problem 2

def problem_2_q(x):
    return (x.T@A_2@x)/2-b_2.T@x

def grad_problem_2_q(x):
    return A_2@x-b_2

def hessian_problem_2_q(x):
    return A_2

#############################################################################################################################################################
# Problem 3

def problem_3_q(x):
    return (x.T@A_3@x)/2-b_3.T@x

def grad_problem_3_q(x):
    return A_3@x-b_3

def hessian_problem_3_q(x):
    return A_3

#############################################################################################################################################################
# Problem 4

def problem_4_q(x):
    return (x.T@A_4@x)/2-b_4.T@x

def grad_problem_4_q(x):
    return A_4@x-b_4

def hessian_problem_4_q(x):
    return A_4

#############################################################################################################################################################
# Problem 5

def problem_5_q(x):
    return (x.T@A_5@x)/2-b_5.T@x

def grad_problem_5_q(x):
    return A_5@x-b_5

def hessian_problem_5_q(x):
    return A_5

<hr><h4 style="background-color: #ADB8FF;"><br>Run 5 tests<br></h4><br>
<p><b>Note:</b> After every test print out the resulsts. 
<br>For your convinience we implemented a function which will do it for you. Function can be used in case after running optimisation you return $x_{optimal}$, and if you have implemented your gradient approximation. Feel free to bring your adjustments.
<br> Additionaly print how many iterations your algotithm needed. You might also provide charts of your taste (if you want).
<p><i>Place for your additional comments and argumentation<i>

In [9]:
def final_printout(x_0,x_optimal,x_appr,f,grad,args,tolerance):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_0: numpy 1D array, corresponds to initial point
    x_optimal: numpy 1D array, corresponds to optimal point, which you know, or have solved analytically
    x_appr: numpy 1D array, corresponds to approximated point, which your algorithm returned
    --------------------------------------------------------------------------------------------------------------
    f: function which takes 2 inputs: x (initial, optimal, or approximated)
                                      **args
       Function f returns a scalar output.
    --------------------------------------------------------------------------------------------------------------
    grad: function which takes 3 inputs: x (initial, optimal, or approximated), 
                                         function f,
                                         args (which are submitted, because you might need
                                              to call f(x,**args) inside your gradient function implementation). 
          Function grad approximates gradient at given point and returns a 1d np array.
    --------------------------------------------------------------------------------------------------------------
    args: dictionary, additional (except of x) arguments to function f
    tolerance: float number, absolute tolerance, precision to which, you compare optimal and approximated solution.
    """
    
    print(f'Initial x is :\t\t{x_0}')
    print(f'Optimal x is :\t\t{x_optimal}')
    print(f'Approximated x is :\t{x_appr}')
    print(f'Is close verificaion: \t{np.isclose(x_appr,x_optimal,atol=tolerance)}\n')
    f_opt = f(x_optimal)
    f_appr = f(x_appr)
    print(f'Function value in optimal point:\t{f_opt}')
    print(f'Function value in approximated point:   {f_appr}')
    print(f'Is close verificaion:\t{np.isclose(f_opt,f_appr,atol=tolerance)}\n')
    print(f'Gradient approximation in optimal point is:\n{grad(x_optimal)}\n')
    grad_appr = grad(x_appr)
    print(f'Gradient approximation in approximated point is:\n{grad_appr}\n')
    print(f'Is close verificaion:\n{np.isclose(grad_appr,np.zeros(grad_appr.shape),atol=tolerance)}')

In [10]:
#your code goes here
print("============================================================================")
print("DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
print("**** PROBLEM 1 ****")
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)
print("============================================================================")
print("**** PROBLEM 2 ****")
x_hat=newton_multi(init, problem_2_q, grad_problem_2_q, hessian_problem_2_q)
final_printout(init, minimizer_2, x_hat, problem_2_q, grad_problem_2_q, {}, 1e-4)
print("============================================================================")
print("**** PROBLEM 3 ****")
x_hat=newton_multi(init, problem_3_q, grad_problem_3_q, hessian_problem_3_q)
final_printout(init, minimizer_3, x_hat, problem_3_q, grad_problem_3_q, {}, 1e-4)
print("============================================================================")
print("**** PROBLEM 4 ****")
x_hat=newton_multi(init, problem_4_q, grad_problem_4_q, hessian_problem_4_q)
final_printout(init, minimizer_4, x_hat, problem_4_q, grad_problem_4_q, {}, 1e-4)
print("============================================================================")
print("**** PROBLEM 5 ****")
x_hat=newton_multi(init, problem_5_q, grad_problem_5_q, hessian_problem_5_q)
final_printout(init, minimizer_5, x_hat, problem_5_q, grad_problem_5_q, {}, 1e-4)

DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES
**** PROBLEM 1 ****
performed 2 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[5.00000000e+00]
 [7.10542736e-15]
 [3.00000000e+00]
 [3.00000000e+00]
 [7.00000000e+00]
 [9.00000000e+00]
 [3.00000000e+00]
 [5.00000000e+00]
 [2.00000000e+00]
 [4.00000000e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[ 8.52651283e-14]
 [-5.68434189e-14]
 [ 5.68434189e-14]
 [-5.68434189e-14]
 [ 2.84217094e-14]
 [ 5.68434189e-14]
 [ 0.00000000e+00]
 [ 2.84217094e-14]
 [ 0.00000000e+00]
 [-2.84217094e

In [11]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, activate_new_stop_crit=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)
print("============================================================================")
print("VERSUS NORMAL STOPPING CRITERION FOR PROBLEM 1")
print("============================================================================")
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, activate_new_stop_crit=False)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 1
*** GRADIENT BELOW TOLERANCE ***
performed 2 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[5.00000000e+00]
 [7.10542736e-15]
 [3.00000000e+00]
 [3.00000000e+00]
 [7.00000000e+00]
 [9.00000000e+00]
 [3.00000000e+00]
 [5.00000000e+00]
 [2.00000000e+00]
 [4.00000000e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[ 8.52651283e-14]
 [-5.68434189e-14]
 [ 5.68434189e-14]
 [-5.68434189e-14]
 [ 2.84217094e-14]
 [ 5.68434189e-14]
 [ 0.000000

In [12]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, activate_alt_gradient=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)
print("============================================================================")
print("VERSUS NORMAL GRADIENT FOR PROBLEM 1")
print("============================================================================")
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, activate_alt_gradient=False)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 1
performed 3 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[ 4.99998655e+00]
 [-9.46895490e-06]
 [ 2.99999595e+00]
 [ 3.00002107e+00]
 [ 7.00000539e+00]
 [ 9.00001754e+00]
 [ 2.99997900e+00]
 [ 5.00000566e+00]
 [ 1.99998676e+00]
 [ 4.00000363e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[-2.04930700e-07]
 [ 1.39831421e-07]
 [-1.66820826e-07]
 [ 3.18845196e-07]
 [ 2.83091879e-07]
 [-3.39930921e-07]
 [-4.31410569e-08]
 [ 2.63258130e-07]
 [

In [13]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED ALTERNATIVE MATRIX INVERSION FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, activate_new_matrix_inversion=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)
print("============================================================================")
print("VERSUS NORMAL MATRIX INVERSION FOR PROBLEM 1")
print("============================================================================")
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, activate_new_matrix_inversion=False)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED ALTERNATIVE MATRIX INVERSION FOR PROBLEM 1
performed 2 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[ 5.00000000e+00]
 [-6.39488462e-14]
 [ 3.00000000e+00]
 [ 3.00000000e+00]
 [ 7.00000000e+00]
 [ 9.00000000e+00]
 [ 3.00000000e+00]
 [ 5.00000000e+00]
 [ 2.00000000e+00]
 [ 4.00000000e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[ 0.00000000e+00]
 [-1.27897692e-13]
 [ 2.84217094e-14]
 [ 8.52651283e-14]
 [-1.13686838e-13]
 [-1.42108547e-13]
 [-1.42108547e-14]
 [-4.26325641e

In [14]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED APPROXIMATED GRADIENT AND STOPPING CRITERION AND ALT MATRIX INVERSION FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, 
                   activate_new_matrix_inversion=True, 
                   activate_alt_gradient=True,
                   activate_new_stop_crit=True
                  )
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)
print("============================================================================")
print("VERSUS NORMAL NEWTON FOR PROBLEM 1")
print("============================================================================")
x_hat=newton_multi(init, problem_1_q, grad_problem_1_q, hessian_problem_1_q, 
                                activate_new_matrix_inversion=False, 
                                activate_alt_gradient=False,
                                activate_new_stop_crit=False
                            )
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED APPROXIMATED GRADIENT AND STOPPING CRITERION AND ALT MATRIX INVERSION FOR PROBLEM 1
*** GRADIENT BELOW TOLERANCE ***
performed 4 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[5.00000003e+00]
 [1.99577286e-08]
 [3.00000001e+00]
 [2.99999995e+00]
 [6.99999999e+00]
 [8.99999996e+00]
 [3.00000005e+00]
 [4.99999999e+00]
 [2.00000003e+00]
 [3.99999999e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[-1.33260869e-09]
 [ 1.21542598e-09]
 [-2.20140350e-09]
 [ 1.62742708e-10]
 [-1.35

<i>Here is some place for your analysis. How the behavour of algorithm changed after adjustments? What are specific details, differences you noticed with respect to other algorithms behaviour.</i>

### ANALYSIS:

Our Newton Algorithm is able to find all global minimizers, which is already really good.<br>
The algorithm is really fast and stable, therefore our additional implemented features are not that important. We can barley see an imporvement in performance.<br>

#### Stopping Criterion:

With the advanced stopping criterion we are able to safe some iterations, depending on the example, but it is not really necessary, since the Newton Method is already really fast.<br>
The results do not get worse by this feature, therefore we can still use it and if we encounter a special example, where the advanced stopping criterion is useful, we still benefit from it.

#### Approximated Gradient:

As we can see above, the approxmation algorithm works perfectly. There is only a small difference between the results. If the difference would be larger, <br>
we would notice that the algorithm converges to infinity and would be able to find the minimizer. Therefore it is really important to find the right tolerance.<br>
Again as already mentioned in the stability section, we have to find the middle way, between low error and fast execution.<br><br>
In this case we do not really see a problem, again not really an improvement, but it also does not get worse. It is still a tradeoff, but Newton works also really well without it.

#### Alternative Matrix Inversion:

The normal Newton Method uses Matrix Inversion and since we know that one should not inverse matrices, because this can cause several problems, we should use an approximation. This approximation solves the mentioned problems and increases the stability of the algorithm. In the examples above, you can see the test of the new matrix inversion and you can see that it functions really well and provides equal results, compared to the built in numpy method. Our examples are not really critical, therefore it does not really make a difference, but it is important to make the algorithm future proof, for examples, which are more difficult to solve.

<h3 style="background-color: #D3D92B;"><br>Testing on functions of 1-2 variables, Non-quadratic objective<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Implement functions to optimise over<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [15]:
#your code goes here
# Stationary points of each problem. Number at the end of abc_X is the problem number. abc indicates the name of the of the 3 stationary points a, b and c.
abc_6 = np.array([1, 20, 300])
abc_7 = np.array([4, 50, 600])
abc_8 = np.array([7, 80, 900])
abc_9 = np.array([1, 100, 200])
abc_10 = np.array([2, 120, 200])


# Function to find the global minimizer from all stationary points, which are given by the function creation
def get_global_minimizer(problem, stat_points):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    problem: function which takes x coordinates to calculate y axis values
    --------------------------------------------------------------------------------------------------------------
    stat_points: array of stationary points, which should be compared
    """
    y = []
    for stat_point in np.array(stat_points).astype(float):
        y.append([problem(stat_point), stat_point])
    y = np.array(y)
    return y[y[:,0].argsort()][0, 1]

#############################################################################################################################################################
# problem_x(x) --> is the problem function f, which is used by algorithm.
# derivative_problem_X(x) --> as the name implies, this function is the derivative of the problem function f (problem_X)
# second_derivative_problem_X(x) --> as the name implies, this function is the second derivative of the problem function f (problem_X)
# in all cases you can parse x coordinates to the functions in order to receive the corresponding y axis values.
#############################################################################################################################################################
# Problem 6

def problem_6(x):
    return -(x*(3*x**3+(-4*abc_6[2]-4*abc_6[1]-4*abc_6[0])*x**2+((6*abc_6[1]+6*abc_6[0])*abc_6[2]+6*abc_6[0]*abc_6[1])*x-12*abc_6[0]*abc_6[1]*abc_6[2]))/12

def derivative_problem_6(x):
    return (abc_6[0]-x)*(abc_6[1]-x)*(abc_6[2]-x)

def second_derivative_problem_6(x):
    return -(abc_6[1]-x)*(abc_6[2]-x)-(abc_6[0]-x)*(abc_6[2]-x)-(abc_6[0]-x)*(abc_6[1]-x)
#############################################################################################################################################################
# Problem 7

def problem_7(x):
    return -(x*(3*x**3+(-4*abc_7[2]-4*abc_7[1]-4*abc_7[0])*x**2+((6*abc_7[1]+6*abc_7[0])*abc_7[2]+6*abc_7[0]*abc_7[1])*x-12*abc_7[0]*abc_7[1]*abc_7[2]))/12

def derivative_problem_7(x):
    return (abc_7[0]-x)*(abc_7[1]-x)*(abc_7[2]-x)

def second_derivative_problem_7(x):
    return -(abc_7[1]-x)*(abc_7[2]-x)-(abc_7[0]-x)*(abc_7[2]-x)-(abc_7[0]-x)*(abc_7[1]-x)

#############################################################################################################################################################
# Problem 8

def problem_8(x):
    return -(x*(3*x**3+(-4*abc_8[2]-4*abc_8[1]-4*abc_8[0])*x**2+((6*abc_8[1]+6*abc_8[0])*abc_8[2]+6*abc_8[0]*abc_8[1])*x-12*abc_8[0]*abc_8[1]*abc_8[2]))/12

def derivative_problem_8(x):
    return (abc_8[0]-x)*(abc_8[1]-x)*(abc_8[2]-x)

def second_derivative_problem_8(x):
    return -(abc_8[1]-x)*(abc_8[2]-x)-(abc_8[0]-x)*(abc_8[2]-x)-(abc_8[0]-x)*(abc_8[1]-x)

#############################################################################################################################################################
# Problem 9

def problem_9(x):
    return -(x*(3*x**3+(-4*abc_9[2]-4*abc_9[1]-4*abc_9[0])*x**2+((6*abc_9[1]+6*abc_9[0])*abc_9[2]+6*abc_9[0]*abc_9[1])*x-12*abc_9[0]*abc_9[1]*abc_9[2]))/12

def derivative_problem_9(x):
    return (abc_9[0]-x)*(abc_9[1]-x)*(abc_9[2]-x)

def second_derivative_problem_9(x):
    return -(abc_9[1]-x)*(abc_9[2]-x)-(abc_9[0]-x)*(abc_9[2]-x)-(abc_9[0]-x)*(abc_9[1]-x)

#############################################################################################################################################################
# Problem 10

def problem_10(x):
    return -(x*(3*x**3+(-4*abc_10[2]-4*abc_10[1]-4*abc_10[0])*x**2+((6*abc_10[1]+6*abc_10[0])*abc_10[2]+6*abc_10[0]*abc_10[1])*x-12*abc_10[0]*abc_10[1]*abc_10[2]))/12

def derivative_problem_10(x):
    return (abc_10[0]-x)*(abc_10[1]-x)*(abc_10[2]-x)

def second_derivative_problem_10(x):
    return -(abc_10[1]-x)*(abc_10[2]-x)-(abc_10[0]-x)*(abc_10[2]-x)-(abc_10[0]-x)*(abc_10[1]-x)

<hr><h4 style="background-color: #ADB8FF;"><br>Run 5 tests<br></h4><br>
<p><i>Place for your additional comments and argumentation<i>

In [16]:
#your code goes here
x_start = np.array([[100]], dtype=float)  # start point for all problems
print("============================================================================")
print("DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES")
print("============================================================================")
print("**** PROBLEM 6 ****")
x_hat = newton_uni(x_start, problem_6,  derivative_problem_6, second_derivative_problem_6)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 7 ****")
x_hat=newton_uni(x_start, problem_7,  derivative_problem_7, second_derivative_problem_7)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_7, abc_7), x_appr=x_hat, f=problem_7, grad=derivative_problem_7, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 8 ****")
x_hat=newton_uni(x_start, problem_8,  derivative_problem_8, second_derivative_problem_8)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_8, abc_8), x_appr=x_hat, f=problem_8, grad=derivative_problem_8, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 9 ****")
x_hat=newton_uni(x_start, problem_9,  derivative_problem_9, second_derivative_problem_9)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_9, abc_9), x_appr=x_hat, f=problem_9, grad=derivative_problem_9, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 10 ****")
x_hat=newton_uni(x_start, problem_10,  derivative_problem_10, second_derivative_problem_10)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_10, abc_10), x_appr=x_hat, f=problem_10, grad=derivative_problem_10, args={}, tolerance=1e-4)

DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES
**** PROBLEM 6 ****
performed 8 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[-0.]]

Is close verificaion:
[[ True]]
**** PROBLEM 7 ****
performed 7 iterations.
Initial x is :		[[100.]]
Optimal x is :		50.0
Approximated x is :	[[50.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-9062500.0
Function value in approximated point:   [[-9062500.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[-0.]]

Is close verificaion:
[[ True]]
**** PROBLEM 8 ****
performed 6 iterations.
Initial x is :		[[100.]]
Optimal x is :		80.0
Approximated x is :	

In [17]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 6")
print("============================================================================")
x_start = np.array([[100]], dtype=float)  # start point for all problems
x_hat = newton_uni(x_start, problem_6,  derivative_problem_6, second_derivative_problem_6, activate_new_stop_crit=True)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)
print("============================================================================")
print("VERSUS NORMAL STOPPING CRITERION FOR PROBLEM 6")
print("============================================================================")
x_hat = newton_uni(x_start, problem_6,  derivative_problem_6, second_derivative_problem_6)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 6
*** GRADIENT SMALLER THAN GRADIENT OF FIRST STEP * TOLERANCE ***
performed 7 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.00000001]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[4.20502021e-05]]

Is close verificaion:
[[ True]]
VERSUS NORMAL STOPPING CRITERION FOR PROBLEM 6
performed 8 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[-0.]]

Is clo

In [18]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 6")
print("============================================================================")
x_start = np.array([[100]], dtype=float)  # start point for all problems
x_hat = newton_uni(x_start, problem_6,  derivative_problem_6, second_derivative_problem_6, activate_alt_gradient=True)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)
print("============================================================================")
print("VERSUS NORMAL GRADIENT FOR PROBLEM 6")
print("============================================================================")
x_hat = newton_uni(x_start, problem_6,  derivative_problem_6, second_derivative_problem_6, activate_alt_gradient=False)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 6
performed 8 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[-1.61101653e-06]]

Is close verificaion:
[[ True]]
VERSUS NORMAL GRADIENT FOR PROBLEM 6
performed 8 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[-0.]]

Is close verificaion:
[[ True]]


In [19]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED APPROXIMATED GRADIENT AND STOPPING CRIT FOR PROBLEM 6")
print("============================================================================")
x_start = np.array([[100]], dtype=float)  # start point for all problems
x_hat = newton_uni(x_start, problem_6,  derivative_problem_6, second_derivative_problem_6, activate_alt_gradient=True, activate_new_stop_crit=True)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)
print("============================================================================")
print("VERSUS NORMAL NEWTON FOR PROBLEM 6")
print("============================================================================")
x_hat = newton_uni(x_start, problem_6,  derivative_problem_6, second_derivative_problem_6, activate_alt_gradient=False)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED APPROXIMATED GRADIENT AND STOPPING CRIT FOR PROBLEM 6
*** GRADIENT SMALLER THAN GRADIENT OF FIRST STEP * TOLERANCE ***
performed 7 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.00000001]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[3.82612126e-05]]

Is close verificaion:
[[ True]]
VERSUS NORMAL NEWTON FOR PROBLEM 6
performed 8 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[-0.]]

Is clo

<i>Here is some place for your analysis. How the behavour of algorithm changed after adjustments? What are specific details, differences you noticed with respect to other algorithms behaviour.</i>

### ANALYSIS:

Our Newton Algorithm is able to find all global minimizers, which is already really good.<br>
The algorithm is really fast and stable, therefore our additional implemented features are not that important. We can barley see an imporvement in performance.<br>

#### Stopping Criterion:

With the advanced stopping criterion we are able to safe some iterations, depending on the example, but it is not really necessary, since the Newton Method is already really fast.<br>
The results do not get worse by this feature, therefore we can still use it and if we encounter a special example, where the advanced stopping criterion is useful, we still benefit from it.

#### Approximated Gradient:

As we can see above, the approxmation algorithm works perfectly. There is only a small difference between the results. If the difference would be larger, <br>
we would notice that the algorithm converges to infinity and would be able to find the minimizer. Therefore it is really important to find the right tolerance.<br>
Again as already mentioned in the stability section, we have to find the middle way, between low error and fast execution.<br><br>
In this case we do not really see a problem, again not really an improvement, but it also does not get worse. It is still a tradeoff, but Newton works also really well without it.

## Comparison:

#### Steepest Descent:

Needs rescaling for unbalanced problem function.<br>
Needs many iterations >20k to find minimizer.<br>
Does not reach lowest error bound, with advanced stopping criterion. This could be fixed, but runtime would be higher.<br>
Stable as long problems are not too complicated (unbalanced, many local min, etc.)<br>

#### Conjugate Gradient:

Needs rescaling for unbalanced problem function.<br>
Needs many iterations >20k to find minimizer.<br>
Does not reach lowest error bound, with advanced stopping criterion. This could be fixed, but runtime would be higher. (Better than steepest descent, still not optimal)<br>
Stable as long problems are not too complicated (unbalanced, many local min, etc.)<br>

#### Newton (CURRENT):

Does not need rescaling for unbalanced problem function. Newton default behaviour.<br>
Really fast <10 iterations for our examples to find solution.<br>
Reaches lowest error bound with and without additional features.<br>
Stable with and without additional features, this could change for more difficult problems, but has not been tested <br>

#### Quasi Netwon:


Does not need rescaling for unbalanced problem function. Newton default behaviour.<br>
Really fast <30 iterations for our examples to find solution.<br>
Reaches lowest error bound with and without additional features.<br>
Stable with and without additional features, this could change for more difficult problems, but has not been tested <br>

<h3 style="background-color: #D3D92B;"><br>Template for teachers' tests<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Set up a template, how one can run your code<br></h4><br>
Template should include sceletons for:<ul>
    <li>custom function to optimise over </li> 
    <li>values initialisation to submit into otimising algorithm </li> 
    <li>optimiser function call</li> 
    <li>report print out call</li> </ul><br>
Provide descriptions and comments.

In [20]:
"""
**** Teacher Test ****
"""

"""
** Custom Function to optimise over
------------------------------------------------------------------------------------------------------------------
- All problems, quadratic and non quadratic are based on the instruction from project phase 1.
*** Quadratic problems
------------------------------------------------------------------------------------------------------------------
- One can easily create new quadratic problems by changing the seed of the "get_random_A_b_minimizer" function. This function uses the seed to create random matrices, which are than used for the problem function.
- The three functions problem_x_q, grad_problem_x and hessian_problem_x are used by the optimizer and those functions use the return values of the get_random_A_b_minimizer functions to create the problem function.
- Therefore one can change the seed in the mentioned function and create a new problem, which can be recreated and is still random.
- It is also possible to create completly different problems, but the structure has to match.
------------------------------------------------------------------------------------------------------------------
*** Non Quadratic problems
------------------------------------------------------------------------------------------------------------------
- One can also create new non quadratic problems by changing the values of the "abc_x" array. Those are the pre defined stationary points of the problem function. By choosing different points, the problem changes.
- The three functions problem_x, derivative_problem_x and second_derivative_problem_x are used by the optimizer and those functions use the abc_x array to create the problem function.
- Therefore one can change the content of the abc_x array and create a new problem. 
- !!!!!!!!!!!!!!!IMPORTANT!!!!!!!!!!!!!!!!!!!!: No additional values should be entered, this means the length of this array should be 3, otherwise the solution might not match.
- It is also possible to create completly different problems, but the structure has to match.
"""

def get_random_A_b_minimizer(seed):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    seed: Seed is an int value, which is used to create reproducable random values.
    --------------------------------------------------------------------------------------------------------------
    - 10x10 random matrix is produced by this function, which is the base of the problem
    - 10x1 random vector is produced, which is also the minimizer of the function and the goal of the algo
    - Function uses thoes variables to create a problem function, which can be solved by the algo
    """
    np.random.seed(seed)
    A=np.random.randint(0, 2, (10, 10))
    A=A@A.T

    np.random.seed(seed)
    minimizer=np.random.randint(0, 10, (10, 1))

    b=A@minimizer

    return A, b, minimizer

# Quadratic problem
A_6, b_6, minimizer_6 = get_random_A_b_minimizer(20)

def problem_6_q(x):
    return (x.T@A_6@x)/2-b_6.T@x

def grad_problem_6_q(x):
    return np.dot(A_6,x)-b_6

def hessian_problem_6_q(x):
    return A_6

#############################################################################################################################################################
# Non quadratic problem

abc_11 = np.array([2, 200, 400])
def problem_11(x):
    return -(x*(3*x**3+(-4*abc_11[2]-4*abc_11[1]-4*abc_11[0])*x**2+((6*abc_11[1]+6*abc_11[0])*abc_11[2]+6*abc_11[0]*abc_11[1])*x-12*abc_11[0]*abc_11[1]*abc_11[2]))/12

def derivative_problem_11(x):
    return (abc_11[0]-x)*(abc_11[1]-x)*(abc_11[2]-x)

def second_derivative_problem_11(x):
    return -(abc_11[1]-x)*(abc_11[2]-x)-(abc_11[0]-x)*(abc_11[2]-x)-(abc_11[0]-x)*(abc_11[1]-x)

"""
--------------------------------------------------------------------------------------------------------------
** Value Initalisation Optimizer & Report Call 
--------------------------------------------------------------------------------------------------------------
- Call function to optimise over and safe result
- Optimizer function is called with "newton_multi" for quadratic problems and "newton_uni" for non quadratic.
- The names come from the number of variables used. There is a strict differnce in our functions between gradient and derivative. It would be possible to use the same function for both problems, but 
- kept it consistent to the individual task
- Decide on a example. 
-- There are 5 quadratic problems named problem_X_q (q for quadratic), the gradient is than named grad_problem_X_q and same for the hessian (X == Number of problem 1..5)
-- There are also 5 non quadratic problems named problem_X, the derivative is than named derivative_problem_X and same for the second derivative (X == Number of problem 6..10)
- We have to also define a starting point for the optimizer. All of our problems, in the quadratic case, have to in shape 10x1, since our quadratic problems have 10 variables
- The non quadratic problems need a 1x1 starting point, since our problems have only 1 variable.
- Additionally one has to choose the extra features. For Newton we implemented the advanced stopping criterion and the approximated gradient and Approximated Inverse (only for Multi). Scaling is not implemented, for reasons checkout the corresponding chapters.
- For the final printout, we use the given function. There is a difference between quadratic and non quadratic problems. Our non quadratic problems have 3 stationary points and we have to call the function
  get_global_minimizer to find the real minimizer. This has to be done for the non quadratic problems.
- x_optimal is for the quadratic problem the variable called "minimizer_X", where X has to be replaced by the problem number (1..5)
- For the non quadratic problems we have an array called "abc_X", where X has to be replaced by the problem number (6..10), which contains all stationary points.
"""
##########################################################################################################################################################################################
# Quadratic problem call

x_start = np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]]) # 10x1 start point for optimizer
problem = problem_6_q # Choose th quadratic problem as problem to be solved by the optimizer
gradient_problem = grad_problem_6_q # Choose 6th quadratic problem gradient to be solved by the optimizer
second_gradient_problem = hessian_problem_6_q # Choose 6th quadratic problem hessian to be solved by the optimizer 
x_optimal = minimizer_6
# We could choose different problems and problem gradients, but that does not make sense, therefore it is not recommended :).
alt_gradient_flag = True # Here one can activate (True), the approximated gradient or deactivate (False) it.
alt_stopping_crit = True # Here one can activate (True), the advanced stopping criterion or deactivate (False) it.
alt_matrix_inv = True # Here one can activate (True), the alternative matrix inversion or deactivate (False) it.

# Optimizer Function Call. All the above defined options are parsed to the optimizer call
x_hat = newton_multi(x_start, problem,  gradient_problem, second_gradient_problem, activate_alt_gradient=alt_gradient_flag, activate_new_stop_crit=alt_stopping_crit, activate_new_matrix_inversion=alt_matrix_inv)
# Final Printout uses the result of the optimizer + the given statements and verifys the results. Args is not used, since our algorithm and functions do not support it.
final_printout(x_0=x_start, x_optimal=x_optimal, x_appr=x_hat, f=problem, grad=gradient_problem, args={}, tolerance=1e-3)
print("==================================================================================================================================")
##########################################################################################################################################################################################
# Non Quadratic problem call

x_start = np.array([[120]], dtype=float) # 1x1 start point for optimizer
problem = problem_11 # Choose 6th non quadratic problem as problem to be solved by the optimizer
gradient_problem = derivative_problem_11 # Choose 6th non quadratic problem derivaitve to be solved by the optimizer
second_gradient_problem = second_derivative_problem_11 # Choose 6th quadratic problem hessian to be solved by the optimizer 
x_optimal = get_global_minimizer(problem_11, abc_11)
# We could choose different problems and problem gradients, but that does not make sense, therefore it is not recommended :).
alt_gradient_flag = False # Here one can activate (True), the approximated gradient or deactivate (False) it.
alt_stopping_crit = True # Here one can activate (True), the advanced stopping criterion or deactivate (False) it.

x_hat = newton_uni(x_start, problem,  gradient_problem, second_gradient_problem, activate_alt_gradient=alt_gradient_flag, activate_new_stop_crit=alt_stopping_crit)
final_printout(x_0=x_start, x_optimal=x_optimal, x_appr=x_hat, f=problem, grad=gradient_problem, args={}, tolerance=1e-3)

*** GRADIENT BELOW TOLERANCE ***
performed 4 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[3]
 [9]
 [4]
 [6]
 [7]
 [2]
 [0]
 [6]
 [8]
 [5]]
Approximated x is :	[[ 2.99999830e+00]
 [ 9.00000432e+00]
 [ 3.99999813e+00]
 [ 5.99999386e+00]
 [ 6.99999888e+00]
 [ 2.00000351e+00]
 [-6.47425291e-07]
 [ 6.00000458e+00]
 [ 8.00000856e+00]
 [ 4.99999418e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-2881.5]]
Function value in approximated point:   [[-2881.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[ 9.93807703e-09]
 [ 2.24970620e-08]
 [-3.21605853e-09]
 [ 3.85455223e-10]
 [-4.27155555e-09]
 [-1.08585141e-10]
 [-3.42416229e-08]
 [-1.43780881e-08]
 [ 1.51937058e-08]
 [ 1.48570223e-09]]

Is close verific